# Minimum Complementary Energy

Over the past two weeks, we have explored the displacement resulting from distributed pressure and the Hertz contact solution. Now, we shift our focus to real contact problems, emphasizing **Variational Principles** and the **Minimum Complementary Energy** approach.

![Contact model discussed in first step](figures/Problem_mapping.png)

Here we simplify our model as the contact between a rigid rough surface and an elastic flat surface[4].



We define our gap function as:

$$
g(x, y)=u_z(x, y)-h(x, y)
$$

where $u_z(x, y)$ is the normal displacement, the separation of two surfaces $h(x,y)$ can sometimes be $h(r)=-\frac{r^2}{2 R}$, like Rey(2017)[5].

In [2]:
import numpy as np
from scipy.optimize import minimize

#define the parameters
F = 1e2  # total force
R = 1  # radius of demi-sphere
L = 2  # domain size
S = L**2  # domain area
p_bar = F / S  # average pressure


#define material parameters



NameError: name 'p' is not defined

In [ ]:

# define the gap function
def g(x, y, R):
    return -(x**2 + y**2) / (2 * R)

# define the complementary energy
def complementary_energy(p, x, y):
    u_z = g(x, y)  # 假设垂直位移等于gap函数
    return np.sum(p * (u_z / 2 + g(x, y)))

# define the constraints
# 平均压力约束??
def constraint_mean_pressure(p):
    return np.sum(p) / S - p_bar

# pressure bounds
bounds = [(0, None) for _ in range(len(p))]

# 初始压力分布假设??
p_initial = np.full((S,), p_bar)

# 定义优化问题??
def objective(p):
    x, y = np.meshgrid(np.linspace(-L/2, L/2, int(np.sqrt(len(p)))), np.linspace(-L/2, L/2, int(np.sqrt(len(p)))))
    x = x.flatten()
    y = y.flatten()
    return complementary_energy(p, x, y)

# 添加约束
constraints = [{'type': 'eq', 'fun': constraint_mean_pressure}]

# 进行优化
result = minimize(objective, p_initial, method='SLSQP', bounds=bounds, constraints=constraints)

if result.success:
    optimized_p = result.x
    print("Optimization successful.")
else:
    print("Optimization failed.")


DOLFINx version: 0.7.3


## Reference:

[1] Frérot, Lucas Henri Galilée. ‘Bridging Scales in Wear Modeling with Volume Integral Methods for Elastic-Plastic Contact’, n.d.

[2] Yastrebov, Vladislav A. Numerical Methods in Contact Mechanics. Numerical Methods in Engineering Series. Hoboken, NJ: Wiley, 2013.

[3] Kalker, J. J. ‘Variational Principles of Contact Elastostatics’. IMA Journal of Applied Mathematics 20, no. 2 (1977): 199–219. https://doi.org/10.1093/imamat/20.2.199.

[4] Vladislav A. Yastrebov, Contact mechanics and elements of tribology, Lecture 4.b, Contact and transport at small scales, Open Course Contact Mechanics and Elements of Tribology, January 23, 2024, https://cmet.yastrebov.fr/index.html

[5] Rey, Valentine, Guillaume Anciaux, and Jean-François Molinari. ‘Normal Adhesive Contact on Rough Surfaces: Efficient Algorithm for FFT-Based BEM Resolution’. Computational Mechanics 60, no. 1 (July 2017): 69–81. https://doi.org/10.1007/s00466-017-1392-5.

